In [1]:
!nvidia-smi
# https://medium.com/analytics-vidhya/install-tensorflow-gpu-2-4-0-with-cuda-11-0-and-cudnn-8-using-anaconda-8c6472c9653f

Tue Mar 09 23:46:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106... WDDM  | 00000000:02:00.0  On |                  N/A |
|  0%   49C    P0    26W / 120W |    426MiB /  6144MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

# Pre-Processing and Training the Model
This notebook goes through the steps in turning the cleaned data into a representation that can be used to train a pre-trained BERT model for sentiment analysis.

__Columns__
- `target` sentiment of tweet (0=positive, 1=negative)
- `tweet` cleaned tweet text

The processes used in this notebook were inspired by the [this tutorial](https://www.tensorflow.org/tutorials/text/classify_text_with_bert) in the TensorFlow Docs.

_Note:_
_This notebook expects the zipped datasets to be in the ./resources sub-directory with the names train_twitter_data.zip, test_twitter_data, and validate_twitter_data (e.g., ./resources/train_twitter_data.zip). If the datasets do not exist open and run the `cleaning.ipynb` notebook to generate them._

In [6]:
import sys 
!conda install --yes --prefix {sys.prefix} tf-nightly
!conda install --yes --prefix {sys.prefix} tensorflow-text-nightly
!conda install --yes --prefix {sys.prefix} tf-models-nightly
!conda install --yes --prefix {sys.prefix} pydot

In [7]:
import os
import shutil

import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from official.nlp import optimization

tf.get_logger().setLevel('ERROR')

AttributeError: module 'tensorflow' has no attribute '__version__'

In [ ]:
# tf.config.set_visible_devices([], 'GPU')
tf.config.list_physical_devices()

## Read Files
Read the files into memory and convert them to TensorFlow datasets to begin pre-processing.

In [ ]:
RESOURCES_PATH = './resources/{}'
BATCH_SIZE=32

def twitter_data_to_dataset(filename):
    pd_df = pd.read_csv(RESOURCES_PATH.format(filename), encoding = 'ISO-8859-1', compression='zip')
    tweets =  tf.constant(pd_df['tweet'].to_numpy(dtype='U'), dtype=tf.string)
    targets = tf.constant(pd_df['target'].to_numpy(dtype='i4'), dtype=tf.int32)
    raw_ds = tf.data.Dataset.from_tensor_slices((tweets, targets))
    return raw_ds.batch(BATCH_SIZE, drop_remainder=False).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds = twitter_data_to_dataset('train_twitter_data.zip')
test_ds = twitter_data_to_dataset('test_twitter_data.zip')
val_ds = twitter_data_to_dataset('validate_twitter_data.zip')

In [ ]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(f'Review: {text_batch.numpy()[i]}')
        print(f'Label : {label_batch.numpy()[i]}')

## Load BERT Resources

In [ ]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

print(f'BERT model           : {tfhub_handle_encoder}')
print(f'Preprocess model     : {tfhub_handle_preprocess}')

### Pre-Processor

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [ ]:
text_test = ['this is a positive tweet']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

### Model

In [ ]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

## Define Model
A fine-tuned model for tweet sentiment analysis will be constructed using the input layer, a pre-processing model, the uncased BERT model, one Dense layer, and a Dropout layer.


In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

In [ ]:
tf.keras.utils.plot_model(classifier_model, RESOURCES_PATH.format('model.png'))

## Train Model

### Loss Function
Tweet sentiment classification between positive and negative is actually a binary classification problem at its core. It outputs a singlue-unit layer in the form of a probability making binary crossentropy the function of choice to measure loss.

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

### Optimizer
Use the same optimizer BERT was trained with the Adam (Adaptive Moments) optimizer.

In [ ]:
EPOCHS = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * EPOCHS
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

### Compile and Train
Now that `loss`, `metrics`, and our `optimizer` are defined we can compile our model and begin training.

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=EPOCHS)